In [2]:
import numpy as np
import pandas as pd


In [3]:
train = pd.read_csv('/mnt/c/users/ventu/Python/Kaggle/House Prices/train.csv')
test = pd.read_csv('//mnt/c/users/ventu/Python/Kaggle/House Prices/test.csv')


In [4]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor

train = pd.read_csv('/mnt/c/users/ventu/Python/Kaggle/House Prices/train.csv')
test = pd.read_csv('/mnt/c/users/ventu/Python/Kaggle/House Prices/test.csv')



In [5]:
xp_train = train.copy()
xp_train.drop
xp_train.GarageYrBlt.fillna(xp_train.YearBuilt, inplace=True)
s = (xp_train.dtypes == 'object')
obj_col = list(s[s].index)

In [6]:
X = xp_train.drop(['SalePrice', 'MiscFeature', 'Alley', 'PoolQC', 'Fence' , 'FireplaceQu'], axis=1)
y = xp_train.SalePrice

numerical_cols = [col for col in X.columns if X[col].dtype in ['int64', 'float64']]

categorical_cols = [col for col in X.columns if X[col].dtype == "object"]

train_X, val_X, train_y, val_y = train_test_split(X, y, train_size=0.8, test_size=0.2)

In [7]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error

numerical_transformer = SimpleImputer(strategy='median')
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_cols),
        ('cat', categorical_transformer, categorical_cols)
    ])

model = RandomForestRegressor(n_estimators=1000, random_state=0)
clf = Pipeline(steps=[('preprocessor', preprocessor),
                      ('model', model)
                     ])
clf.fit(train_X, train_y)
preds = clf.predict(val_X)

In [8]:
print(mean_absolute_error(val_y, preds))
print(clf.score(val_X,val_y))

17841.74954452055
0.8538167461181141


In [9]:
from xgboost import XGBRegressor

model_2 = XGBRegressor(n_estimators=1000)
model_2_pl = Pipeline(steps=[('preprocessor', preprocessor),
                            ('model_2',model_2)])

model_2_pl.fit(train_X, train_y)
prediction = model_2_pl.predict(val_X)

print(mean_absolute_error(val_y, prediction))
print(model_2_pl.score(val_X,val_y))


17524.218121254282
0.8867700668980507
